In [ ]:
#https://medium.com/turing-talks/extraindo-tweets-da-elei%C3%A7%C3%A3o-municipal-de-s%C3%A3o-paulo-com-python-e-tweepy-c55e63c4ccd3

#https://dev.to/mchdax/coletando-dados-do-twitter-com-python-1c7k



'''
ver se esta mesmo
pip3 install tweepy==3.10.0

'''


In [8]:
import tweepy
import os
import json


with open('xina.json') as config_file:
    data = json.load(config_file)



#credenciais da API do twitter
c_key = data['API_KEY']
c_secret = data['API_KEY_SECRET']
a_token = data['ACCESS_TOKEN']
a_token_secret = data['ACCESS_TOKEN_SECRET']

#incializando nossa conexão com a API do twitter
auth = tweepy.OAuthHandler(c_key, c_secret)
auth.set_access_token(a_token,a_token_secret)
api = tweepy.API(auth)

In [9]:
query = "(Bolsonaro OR Lula)"
tweets = []
busca = api.search(query,count=100,tweet_mode="extended")
for tweet in busca:
    tweet = tweet._json
    tweets.append(tweet)

TweepError: [{'message': 'You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve', 'code': 453}]

In [1]:
#pip install -U textblob
import re
import tweepy
from datetime import datetime
from textblob import TextBlob

CONSUMER_KEY = '8GBdcK5UWL4YyOcYt9HBNVT1a'
CONSUMER_SECRET = 'gNkGecdJEdcSPDt3iIXfvOiUInGTb94QuCe5ZQBg2nFdSP6BhL'
ACCESS_TOKEN = '1480522199123562497-qcIVy16S7NFwg9kpThQpSnCBIPqlwQ'
ACCESS_TOKEN_SECRET = 'KbRT2WjjwOx7WUrLZDPA4dIK8iGohiWhoobLSzxQoG6sR'

keyword = ("Bolsonaro OR Lula")
count = 5000

class TweetAnalyzer():

    def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret):
        '''
          Conectar com o tweepy
        '''
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)

        self.conToken = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, retry_count=5, retry_delay=10)

    def __clean_tweet(self, tweets_text):
        '''
        Tweet cleansing.
        '''

        clean_text = re.sub(r'RT+', '', tweets_text) 
        clean_text = re.sub(r'@\S+', '', clean_text)  
        clean_text = re.sub(r'https?\S+', '', clean_text) 
        clean_text = clean_text.replace("\n", " ")

        return clean_text

    def search_by_keyword(self, keyword, count=10, result_type='mixed', lang='en', tweet_mode='extended'):
        '''
          Search for the twitters thar has commented the keyword subject.
        '''
        tweets_iter = tweepy.Cursor(self.conToken.search,
                              q=keyword, tweet_mode=tweet_mode,
                              rpp=count, result_type=result_type,
                              since=datetime(2020,7,31,0,0,0).date(),
                              lang=lang, include_entities=True).items(count)

        return tweets_iter

    def prepare_tweets_list(self, tweets_iter):
        '''
          Transforming the data to DataFrame.
        '''

        tweets_data_list = []
        for tweet in tweets_iter:
            if not 'retweeted_status' in dir(tweet):
                tweet_text = self.__clean_tweet(tweet.full_text)
                tweets_data = {
                    'len' : len(tweet_text),
                    'ID' : tweet.id,
                    'User' : tweet.user.screen_name,
                    'UserName' : tweet.user.name,
                    'UserLocation' : tweet.user.location,
                    'TweetText' : tweet_text,
                    'Language' : tweet.user.lang,
                    'Date' : tweet.created_at,
                    'Source': tweet.source,
                    'Likes' : tweet.favorite_count,
                    'Retweets' : tweet.retweet_count,
                    'Coordinates' : tweet.coordinates,
                    'Place' : tweet.place
                }

            tweets_data_list.append(tweets_data)

        return tweets_data_list

    def sentiment_polarity(self, tweets_text_list):
        tweets_sentiments_list = []
        
        for tweet in tweets_text_list:
            polarity = TextBlob(tweet).sentiment.polarity
            if polarity > 0:
                tweets_sentiments_list.append('Positive')
            elif polarity < 0:
                tweets_sentiments_list.append('Negative')
            else:
                tweets_sentiments_list.append('Neutral')
        return tweets_sentiments_list


In [2]:
analyzer = TweetAnalyzer(consumer_key = CONSUMER_KEY, consumer_secret = CONSUMER_SECRET, access_token = ACCESS_TOKEN, access_token_secret=ACCESS_TOKEN_SECRET)

# Para realizar as buscas usando a keyword e quantidade predefinida e converter em uma lista.
tweets_iter = analyzer.search_by_keyword(keyword, count)
tweets_list = analyzer.prepare_tweets_list(tweets_iter)

# Uso de um Dataframe para melhor manipulação
tweets_df = pd.DataFrame(tweets_list)

# Chamar a função para análise de sentimentos
tweets_df['Sentiment'] = analyzer.sentiment_polarity(tweets_df['TweetText'])

TweepError: Twitter error response: status code = 403

In [5]:
tweets_iter

In [13]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
  
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = '8GBdcK5UWL4YyOcYt9HBNVT1a'
        consumer_secret = 'gNkGecdJEdcSPDt3iIXfvOiUInGTb94QuCe5ZQBg2nFdSP6BhL'
        access_token = '1480522199123562497-qcIVy16S7NFwg9kpThQpSnCBIPqlwQ'
        access_token_secret = 'KbRT2WjjwOx7WUrLZDPA4dIK8iGohiWhoobLSzxQoG6sR'
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

            


In [14]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'Donald Trump', count = 200)
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])

        


In [15]:
main()

Error : [{'message': 'You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve', 'code': 453}]


TypeError: 'NoneType' object is not iterable

In [17]:
import matplotlib.pyplot as plt
%matplotlib inline
import re 
import pandas as pd
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import csv
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
n_words= set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from wordcloud import WordCloud,STOPWORDS
porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osmar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Osmar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Osmar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
def TwitterClient(): 
      # keys and tokens from the Twitter Dev Console 
      consumer_key = 'xxxxxxxx'
      consumer_secret ="xxxxxxxxxxx"
      access_token = 'xxxxxxxxxx'
      access_token_secret = 'xxxxxxxxxxx'

      # attempt authentication 
      try: 
          # create OAuthHandler object 
          auth = OAuthHandler(consumer_key, consumer_secret) 
          # set access token and secret 
          auth.set_access_token(access_token, access_token_secret) 
          # create tweepy API object to fetch tweets 
          api = tweepy.API(auth) 
      except: 
          print("Error: Authentication Failed") 

      return api